The purpose of this tutorial is to find a good solution for the [traveling salesman problem](https://en.wikipedia.org/wiki/Travelling_salesman_problem) by using an evolutionary algorithm. To do this, you can use the functions used in the N-queens problem, or add your own. The notebook of the N-queens problem describes several options for crossover, mutation, and parent selection.

By 3:45 pm, send a .txt file with the best solution you found to annemarijn@pipple.nl. The format for the solution and how to write it to the correct format is already described in this notebook.

---

We use a dataset of all places in Oman, the distances are symmetric (from A to B is as far as from B to A). We have already done some preliminary work, so that you will spend less time on data preprocessing and all kinds of boundary conditions, so that you can focus as much as possible on the evolutionary algorithm. Below, the data is read in from github.

In [ ]:
url = "https://raw.githubusercontent.com/LarsPipple/Pipple_lecture/master/TSP_Oman.txt"

import pandas as pd
X = pd.read_csv(url, sep = ' ', header = None)
X.columns = ['ID', 'X', 'Y']
N = X.shape[0]

Obviously, we need the distance matrix to do calculations. The input data are coordinates, so we can calculate the distance between place i and j with the Pythagorean theorem (Euclidean distance). Since there are quite some cities, the code below may take a while.

In [ ]:
import numpy as np
import math

v = X[['X','Y']].values
dist = lambda p1, p2: math.sqrt(((p1-p2)**2).sum())
distance_matrix = np.zeros((N,N))
distance_matrix = np.asarray([[dist(p1, p2) for p2 in v] for p1 in v])

In addition, we would like to know how long a given route is. For this you can use the function below.

In [ ]:
def getObjValue(solution):
    objective_value = 0
    for i in range(N - 1):
        # Sum the distance from i to i + 1
        objective_value += distance_matrix[solution[i]][solution[i+1]]
    # Add the distance from the last city back to the first
    objective_value += distance_matrix[solution[N - 1]][solution[0]]
    return objective_value

An example solution is to visit all the cities in the order of how they appear in the dataset.

In [ ]:
solution = list(range(N))

Below is a function to plot a solution on a map.

In [ ]:
import matplotlib.pyplot as plt

def plotSolution(solution):
    # Plot the cities
    plt.plot(X['Y'], X['X'], ',')
    # Plot the lines from the current solution
    for i in range(N - 1):
            plt.arrow(X.iloc[solution[i]]['Y'], X.iloc[solution[i]]['X'],
                     (X.iloc[solution[i+1]]['Y'] - X.iloc[solution[i]]['Y']),
                     (X.iloc[solution[i+1]]['X'] - X.iloc[solution[i]]['X']))
    plt.arrow(X.iloc[solution[N - 1]]['Y'], X.iloc[solution[N - 1]]['X'],
             (X.iloc[solution[0]]['Y'] - X.iloc[solution[N - 1]]['Y']),
             (X.iloc[solution[0]]['X'] - X.iloc[solution[N - 1]]['X']))
    plt.xlabel('X-coordinate')
    plt.ylabel('Y-coordinate')
    plt.show()

What does our solution look like on the map? And what distance does our route have?

In [ ]:
print(f"Solution value: {getObjValue(solution)}")
plotSolution(solution)

It may seem that this is not a solution to the problem, but that there are many more lines. However, this is not the case: in the data set there are a lot of coordinates that are very close to each other, which makes it look like a city is visited more than once. However, that there is room for improvement is clear: That is up to you! You can use the notebook of the N-queens problem as a source of inspiration (e.g. the class individual is also suitable for this algorithm), but for a really good and personal algorithm, you can of course also add new functions to the evolutionary algorithm! Below already a framework you can use as a starting point for your algorithm.

First of all, a class for creating individual solutions. In principle nothing needs to be changed, although the initial generation of solutions is now completely random. Here you could possibly adjust things.

In [ ]:
import random as rd

# Generate class individuals
class Individual:
    def __init__(self, length, genes = None):
        self.length = length
        self.fitness = None
        if genes is not None:
            self.genes = genes
        else:
            self.genes = []*length


    # Randomly initialize genes                                   --- Own input possible
    def Initialise(self):
        self.genes = rd.sample(range(self.length), self.length)

    # Create a fitness function to calculate how "good" the current individual is
    # (is identical to getObjValue(solution) of above)
    def GetFitness(self, N):
        """
        Calculate an individual's fitness
        :param self: individual
        """
        objective_value = 0
        for i in range(N-1):
            # Sum the distance from i to i + 1
            objective_value += distance_matrix[self.genes[i]][self.genes[i+1]]
        # Add the distance from the last city back to the first
        objective_value += distance_matrix[self.genes[N - 1]][self.genes[0]]
        self.fitness = objective_value

    # Function to be able to iterate over the class
    def __getitem__(self, key):
        return self.genes[key]

Parent selection: Select a group of parents from the current population. Here the same method is used as in the N-queens problem: Tournament Selection. This will work just fine but another method of parent selection might work better, like fitness proportional selection or rank based selection.

In [ ]:
def TournamentSelection(pop, size, tournament_size):
    """
    Creates a population of parents based on tournament selection: tournament_size
    individuals are selected from pop and the one with the highest fitness wins
    and is added to the parent population.
    :param pop: The population used to select parents from
    :param size: the size of the parent population
    :param tournament_size: The size of a tournament. Must be smaller than population_size
    :return: the parent population
    """
    parent_pop = []
    for i in range(size):
        # select tournament_size individuals (random sampling without replacement)
        tournament = rd.sample(pop, tournament_size)
        # select the best individual
        winner = np.argmin([i.fitness for i in tournament])
        # add it to parent_pop
        parent_pop.append(pop[winner])
    return(parent_pop)

Cross over: Generate a child based on two parents. This is done in the same way as for the N-queens problem. Other methods are also possible here, google is your friend ;)

In [ ]:
def CrossOver(parent1, parent2, k, l):
    """
    crossover applied to two parents
    :param parent1: parent 1
    :param parent2: parent 2
    :param k l: breakpoints of the genen. Before k will be used of parent1, afterwards of parent2
    :return: a child with combined genes
    """

    child = []
    # Select the to-be-used genes of both parents
    childcenter = [parent1.genes[j] for j in range(k,l+1)]
    childedges = [parent2.genes[j] for j in range(0,N) if parent2.genes[j] not in childcenter]
    # Combine the genes
    child[:k] = childedges[:k]
    child[k:l+1] = childcenter
    child[l+1:] = childedges[k:]

    return(child)

Child mutation: Mutate a child in a certain way. Currently swap is used, as in the N-queens problem. But you could also use k-opt, scramble or insert.

In [ ]:
def Swap(child, size):
    """
    Swap two genes
    :param child: child to be mutated
    :param factor: in (1,2), will we swap one or two genes?
    :return: mutated child
    """

    for i in range(size):
      # do a swap
      gene1 = rd.sample(range(N),1)[0]
      gene2 = rd.sample(range(N),1)[0]
      old = child[gene1]
      child[gene1] = child[gene2]
      child[gene2] = old

    return(child)

Get best: obtain the best current solution from the population. In this, in principle, nothing needs to be changed.

In [ ]:
def GetBest(population):
    """
    :param: the current population
    :return: return the best individual of the current population
    """
    best = np.argmin([i.fitness for i in population])
    return(population[best])

The core of the code: calling all the functions. In this, anything you want can be modified. What should be adapted in any case is the while loop: when are you satisfied with the solution found?

In [ ]:
# Global parameters
population_size = 10  # Input parameter which can be changed

# You could add other global parameters
tournament_size = 4
parent_size = 6
mutation_factor = 1
children_size = 50

# Initialize the population
population = [Individual(N) for i in range(population_size)]
for i in population:
    # Initialize randomly and calculate each individual's fitness
    i.Initialise()
    i.GetFitness(N)
population[0] = Individual(N,genes=solution)
population[0].GetFitness(N)

# Evolutionary cycle

i = 0
#while not Stopping condition:
while GetBest(population).fitness > 100000:   # This stopping condition can be adjusted
    # Select parents
    parents = TournamentSelection(population, parent_size, tournament_size)

    childs = []

    # Transfer the best children to the childrenset
    childs.append(GetBest(population))

    while len(childs) < children_size:

        # Select parents from the parent set
        parent1 = rd.sample(parents, 1)[0]
        parent2 = rd.sample(parents, 1)[0]

        # Create children by applying crossover
        (a,b) = rd.sample(range(N),2)
        k = min(a,b)
        l = max(a,b)
        child1 = CrossOver(parent1, parent2, k, l)
        child2 = CrossOver(parent2, parent1, k ,l)

        # Apply mutation to those children
        child1 = Swap(child1, mutation_factor)
        child2 = Swap(child2, mutation_factor)

        # Update children
        childs.append(Individual(N,genes=child1))
        childs.append(Individual(N,genes=child2))

    # Update population
    # Calculate fitness
    [i.GetFitness(N) for i in childs]
    idx = np.argpartition([i.fitness for i in childs], population_size)
    index = [i for i in idx[:population_size]]
    population = [childs[i] for i in index]

    i = i + 1
    if i%10 == 0:
      print(str(GetBest(population).fitness) + " after iteration " + str(i))


GetBest(population)

print("Solution found")

Please send your solution in .txt format to annemarijn@pipple.nl no later than 3:45pm. You can use the function below to get your solution to .txt format.

In [ ]:
from google.colab import files
def writetxt(solution, team_name = 'name'):
    with open(team_name + '.txt', 'w') as f:
        for i in solution:
            f.write("%s\n" % i)
    f.close()
    files.download(team_name + '.txt')

In [ ]:
writetxt(GetBest(population), team_name = "test2")